In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 라이브러리 버젼 체크 : 충돌방지

In [2]:
import numpy as np
np.__version__

'2.0.2'

In [3]:
import sklearn
sklearn.__version__

'1.7.1'

In [4]:
import pandas as pd
pd.__version__

'2.3.0'

# 데이터 체크

In [5]:
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition_변환.csv')
df

,나이,회사와의마찰,출장,일당,부서,거리,학력,전공,사번,만족도,...,성인여부,야근,업무평가,주변평가,근무기준시간,스톡옵션레벨,경력,전년도교육출장횟수,워라밸,현회사근속년수
0,41,Yes,1~29회,1102,영업,1,2,사회과학계열,1,2,...,Y,Yes,보통,1,80,0,8,0,1,6
1,49,No,30회 이상,279,R&D,8,1,사회과학계열,2,3,...,Y,No,좋다,4,80,1,10,3,3,10
2,37,Yes,1~29회,1373,R&D,2,2,기타,4,4,...,Y,Yes,보통,2,80,0,7,3,3,0
3,33,No,30회 이상,1392,R&D,3,4,사회과학계열,5,4,...,Y,Yes,보통,3,80,0,8,3,3,8
4,27,No,1~29회,591,R&D,2,1,자연과학계열,7,1,...,Y,No,보통,4,80,1,6,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,30회 이상,884,R&D,23,2,자연과학계열,2061,3,...,Y,No,보통,3,80,1,17,3,3,5
1466,39,No,1~29회,613,R&D,6,1,자연과학계열,2062,4,...,Y,No,보통,1,80,1,9,5,3,7
1467,27,No,1~29회,155,R&D,4,3,사회과학계열,2064,2,...,Y,Yes,좋다,2,80,1,6,0,3,6
1468,49,No,30회 이상,1023,영업,2,3,자연과학계열,2065,4,...,Y,No,보통,4,80,0,17,3,2,9


In [6]:
df[df.isna().any(axis=1)] # 결측치 0건

,나이,회사와의마찰,출장,일당,부서,거리,학력,전공,사번,만족도,...,성인여부,야근,업무평가,주변평가,근무기준시간,스톡옵션레벨,경력,전년도교육출장횟수,워라밸,현회사근속년수


In [7]:
df.columns

Index(['나이', '회사와의마찰', '출장', '일당', '부서', '거리', '학력', '전공', '사번', '만족도', '성별',
       '시급', '참여프로젝트', '근속연차', '직급', '직업만족도', '결혼여부', '월급', '이직회수', '성인여부',
       '야근', '업무평가', '주변평가', '근무기준시간', '스톡옵션레벨', '경력', '전년도교육출장횟수', '워라밸',
       '현회사근속년수'],
      dtype='object')

In [8]:
x_col = ['나이','출장','부서','학력','전공','참여프로젝트','직급','경력','전년도교육출장횟수','현회사근속년수']
y_col = ['업무평가']

# 데이터 전처리

In [9]:
# 데이터 우선 재체크
df_data = df[x_col + y_col]
X = df_data[x_col]
y = df_data[y_col]
X.shape, y.shape

((1470, 10), (1470, 1))

# Ordinal Encoding : 출장횟수 분류, 직급

In [10]:
print(X['출장'].unique()) # [1,2,0]
print(X['직급'].unique()) # {"영업직" : 0, "연구직" : 0, "인사사원" : 0, "엔지니어":0, "제조책임자":1,"연구직관리자":1, "인사관리자":1,"영업담당자":1,"연구관리자":2}

['1~29회' '30회 이상' '0회']
['영업직' '연구직' '엔지니어' '제조 책임자' '연구직 관리자' '인사관리자' '영업 담당자' '연구 관리자' '인사사원']


In [11]:
def ordinalEncoding(row) : 
    row = row.copy()
    manager_dict = {"영업직" : 0, "연구직" : 0, "인사사원" : 0, "엔지니어":0, "제조 책임자":1,"연구직 관리자":1, "인사관리자":1,"영업 담당자":1,"연구 관리자":2}
    if row['출장'] == "1~29회" :
        row['출장odm'] = 1
    elif row['출장'] == "30회 이상" :
        row['출장odm'] = 2
    else :
        row['출장odm'] = 0
    row['직급odm'] = manager_dict.get(row['직급'])
    return row

ordinalEncoding(X.iloc[0]) # ok

나이               41
출장            1~29회
부서               영업
학력                2
전공           사회과학계열
참여프로젝트            3
직급              영업직
경력                8
전년도교육출장횟수         0
현회사근속년수           6
출장odm             1
직급odm             0
Name: 0, dtype: object

In [12]:
X_odm = X.apply(ordinalEncoding, axis=1,)
X_odm

,나이,출장,부서,학력,전공,참여프로젝트,직급,경력,전년도교육출장횟수,현회사근속년수,출장odm,직급odm
0,41,1~29회,영업,2,사회과학계열,3,영업직,8,0,6,1,0
1,49,30회 이상,R&D,1,사회과학계열,2,연구직,10,3,10,2,0
2,37,1~29회,R&D,2,기타,2,엔지니어,7,3,0,1,0
3,33,30회 이상,R&D,4,사회과학계열,3,연구직,8,3,8,2,0
4,27,1~29회,R&D,1,자연과학계열,3,엔지니어,6,3,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,30회 이상,R&D,2,자연과학계열,4,엔지니어,17,3,5,2,0
1466,39,1~29회,R&D,1,자연과학계열,2,연구직 관리자,9,5,7,1,1
1467,27,1~29회,R&D,3,사회과학계열,4,제조 책임자,6,0,6,1,1
1468,49,30회 이상,영업,3,자연과학계열,2,영업직,17,3,9,2,0


In [13]:
X_odm[X_odm.isna().any(axis=1)] # 결측치 0건 : 정상작동

,나이,출장,부서,학력,전공,참여프로젝트,직급,경력,전년도교육출장횟수,현회사근속년수,출장odm,직급odm


In [27]:
import numpy as np
x_cat_nominal = ['전공','직급']
x_recol = ['나이','학력','참여프로젝트','경력','전년도교육출장횟수','현회사근속년수','출장odm','직급odm'] + x_cat_nominal
X_odm['전공'].astype('category')
X_odm['직급'].astype('category')

0           영업직
1           연구직
2          엔지니어
3           연구직
4          엔지니어
         ...   
1465       엔지니어
1466    연구직 관리자
1467     제조 책임자
1468        영업직
1469       엔지니어
Name: 직급, Length: 1470, dtype: category
Categories (9, object): ['엔지니어', '연구 관리자', '연구직', '연구직 관리자', ..., '영업직', '인사관리자', '인사사원', '제조 책임자']

In [28]:
from sklearn.preprocessing import LabelEncoder
le_major = LabelEncoder()
le_manager = LabelEncoder()

In [29]:
X_odm['전공'] = le_major.fit_transform(X_odm['전공'])
X_odm['직급'] = le_manager.fit_transform(X_odm['직급'])

In [30]:
X_value = X_odm[x_recol].values
X_value.shape, y.shape

((1470, 10), (1470, 1))

In [31]:
# 학습셋, 시험셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_value,y,test_size=0.2,stratify=y)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1176, 10), (294, 10), (1176, 1), (294, 1))

## 단조 제약 고려 :
- 단조 제약은 “출장↑/직급↑이면 평가가 대체로 좋아진다” 같은 비즈니스 상식이 맞을 때 작동 시켜보기

# 모델 : LightGBM
- 범주형 변수는 lgbm 네이티브 범주형 사용

In [32]:
np.array(X_train)

array([[38,  3,  3, ...,  1,  5,  3],
       [38,  3,  3, ...,  1,  5,  8],
       [36,  2,  3, ...,  0,  2,  0],
       ...,
       [35,  4,  3, ...,  1,  5,  3],
       [35,  1,  3, ...,  1,  2,  3],
       [35,  1,  4, ...,  0,  2,  0]])

In [33]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(
    objective= 'binary',    
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    min_data_in_leaf=20,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42
)

# 모델 학습

In [35]:
history = lgbm.fit(X_train, Y_train,)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Number of positive: 181, number of negative: 995
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 150
[LightGBM] [Info] Number of data points in the train set: 1176, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.153912 -> initscore=-1.704246
[LightGBM] [Info] Start training from score -1.704246
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# 모델 결과

In [36]:
pred = lgbm.predict(np.array(X_test))
value = np.array(Y_test).ravel()

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [37]:
pred.shape, value.shape

((294,), (294,))

In [38]:
pd.crosstab(value,pred, rownames=['실제값'],colnames=['예측값'])

예측값,보통,좋다
실제값,,
보통,232,17
좋다,39,6


In [40]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(value, pred))
print("accu",accuracy_score(value,pred))

              precision    recall  f1-score   support

          보통       0.86      0.93      0.89       249
          좋다       0.26      0.13      0.18        45

    accuracy                           0.81       294
   macro avg       0.56      0.53      0.53       294
weighted avg       0.76      0.81      0.78       294

accu 0.8095238095238095


# LightGBM 결과 정확도가 80%로 약 4.7% 가량 감소한 형태
- 그러나 좋다 범주에 대한 precision은 0.26으로 현재까지 모델 중 가장 좋은 값 기록